In [1]:
import os


In [2]:
%pwd

'c:\\Users\\MAINGATE\\desktop\\Maize_disease_classification\\end-to-end_mazie_disease_classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\MAINGATE\\desktop\\Maize_disease_classification\\end-to-end_mazie_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    source_PATH: Path


In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'joblib'

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config= DataIngestionConfig(
          root_dir=config.root_dir,
          source_URL= config.source_URL,
          source_PATH= config.source_PATH,
          local_data_file=config.local_data_file,
          unzip_dir= config.unzip_dir,
          #s3_bucket_name=getattr(config, "s3_bucket_name", None),
          #s3_file_key=getattr(config, "s3_file_key", None)

    )

        return data_ingestion_config


In [ ]:
import os
import urllib.request as request
import zipfile
import shutil
import mmap
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [53]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config

    def download_file(self):

        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.url.retrieve(
            url=self.config.source_URl,
            filname= self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
                      
    """def download_s3_file(self):
          
          #Note to use the s3 sdk  install the python 3.7 version for compatibility purpose
          
          if not os.path.exists(self.config.local_data_file):
              s3= boto3.client('s3')
              s3.download_file(
                  self.config.s3_bucket_name, 
                  self.config.s3_file_key,
                  self.config.local_data_file)
              logger.info(f"File {self.config.file_key} downloaded successfully from S3 bucket{self.config.bucket_name}!")
          else:
              logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")"""   

    def copy_file(self):

        if not os.path.exists(self.config.local_data_file):
            with open(self.config.source_PATH, 'rb') as src, open(self.config.local_data_file, 'wb') as dst:
                src_mmap = mmap.mmap(src.fileno(), 0, access=mmap.ACCESS_READ)
                dst.write(src_mmap)
                src_mmap.close()

            logger.info(f"File copied from {self.config.source_PATH} to {self.config.local_data_file}")


    def extract_zip_files(self):

        """
        zip_file_path:str
        Extract the zip file into the data directory
        function returns None
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [54]:
# if you intend to download the file from a url
"""try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_files()

except Exception as e:
    raise e
    """

'try:\n    config=ConfigurationManager()\n    data_ingestion_config=config.get_data_ingestion_config()\n    data_ingestion=DataIngestion(config=data_ingestion_config)\n    data_ingestion.download_file()\n    data_ingestion.extract_zip_files()\n\nexcept Exception as e:\n    raise e\n    '

In [55]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.copy_file()
    data_ingestion.extract_zip_files()

except Exception as e:
    raise e

[2025-04-12 15:29:07,211: INFO: common: yaml file: (path_to_yaml) loaded successfully]
[2025-04-12 15:29:07,221: INFO: common: yaml file: (path_to_yaml) loaded successfully]
[2025-04-12 15:29:07,228: INFO: common: created a directory at : artifacts]
[2025-04-12 15:29:07,234: INFO: common: created a directory at : artifacts/data_ingestion]
[2025-04-12 15:43:20,267: INFO: <ipython-input-53-6bdb4a664f34>: File copied from C:/Users/MAINGATE/Downloads/Maize_leaf_dataset.zip to artifacts/data_ingestion/data.zip]
